In [1]:
import data_struct as dm

import numpy as np
import pandas as pd
import time
#import asyncio

In [4]:
filepath = "S:\\Sync\\University\\2023_MRP_1\\MRP1_WorkDir\\data\\"

dm.Scraper.query_from_subreddits(query="quantum self:no",
                                 subreddits=

#Convert to a standardized object structure before exporting to csv files:
artset = dm.ArticleSet.from_reddit_dfs(artset_descript="Test Reddit ArticleSet", 
                                    post_df = post_df, 
                                    comment_df = comment_df,
                                    scraper=scraper,)

#print(artset.as_list[0].comments[2])

artset.save_to_csv(filepath)


In [ ]:
comment_df["depth"]


In [ ]:
#OR store directly in csv files via a direct mapping:
reddit_postFeatures = ["id","ext_link_url"]
reddit_commentFeatures = ["id", "parent_post_id", "created_utc", "body", "depth","parent_comment_id" , "replies_ids"]

target_articleFeatures = ["article_id", "article_type", "link", "date", "author", "headline", "body", "char_count", "comments_ids"] # [field.name for field in fields(Article)].remove("comments")
target_commentFeatures =  ["comment_id", "article_id", "date", "username", "from_where" ,"body", "level" , "parent_comment_id", "replies_ids"]
direct_target_articleFeatures = target_articleFeatures.copy().remove("article_type", "date", "author", "headline", "body", "char_count", "comments_ids") #Subset of target_articleFeatures which can be directly mapped from the reddit data
direct_target_commentFeatures = target_commentFeatures.copy() #Subset of target_commentFeatures which can be directly mapped from the reddit data

reddit_to_commentdf_map = dict(zip(direct_target_articleFeatures, reddit_postFeatures))
reddit_to_articledf_map = dict(zip(direct_target_commentFeatures, reddit_commentFeatures))

export_article_df = pd.DataFrame(columns=target_articleFeatures)
export_comment_df = pd.DataFrame(columns=target_commentFeatures)
export_article_df.loc[:, reddit_to_articledf_map.keys()] = post_df.loc[:, reddit_to_articledf_map.values()]
export_comment_df.loc[:, reddit_to_commentdf_map.keys()] = comment_df.loc[:, reddit_to_commentdf_map.values()]

export_article_df.loc[:,"article_type"] = "web_article"
#Parse: "date", "author", "headline", "body" from ext link url
# 
export_article_df.loc[:,"char_count"] = len(export_article_df.loc[:,"body"]) 
# Map "comments_ids" to the articles


In [2]:
from trafilatura import fetch_url, extract, bare_extraction
from courlan import validate_url, check_url, clean_url
import requests
import re
from datetime import datetime

#url = 'https://github.blog/2019-03-29-leader-spotlight-erin-spiceland/'
url = "https://scottaaronson.blog/?p=6871"
#url = "https://tweakers.net/nieuws/189740/qutech-start-quantum-network-explorer-voor-publieke-experimenten-quantumnetwerk.html"
#url = "https://www.quantamagazine.org/how-quantum-physicists-flipped-time-and-how-they-didnt-20230127/"
#url = "https://news.st-andrews.ac.uk/archive/st-andrews-scientists-turn-up-the-heat-on-physics-phenomenon/#:~:text=A%20'quantum%20harmonic%20oscillator'%20%E2%80%93,the%20University%20of%20St%20Andrews."
#url = "https://www.vice.com/en/article/88qj3z/government-scientists-discover-entirely-new-kind-of-quantum-entanglement-in-breakthrough"
#url = "https://www.nature.com/articles/s41534-022-00631-2"
#url = "https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiX073B8vH8AhUThP0HHe5FDd8QtwJ6BAgSEAI&url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DjHoEjvuPoB8&usg=AOvVaw2oWlY57R1_JHg0bgTgPkoW"
#url = "https://youtu.be/jHoEjvuPoB8"
#url = "https://www.scmp.com/news/china/article/1631479/china-launch-hack-proof-quantum-communication-network-2016?page=all"


assert validate_url(url)[0] == True #check whether url can be resolved or is damaged

url = clean_url(url)
downloaded = fetch_url(url)

result = bare_extraction(downloaded, 
                            url=url, 
                            include_comments=True,
                            #include_formatting=False,
                            favor_precision=True, #favor recall
                            with_metadata=True,
                            #deduplicate=True, 
                            only_with_metadata=True,
                            #url_blacklist=None,
                            #as_dict=True,
                            )

#print(result.keys())
print(result["title"], "...", result["author"],"...",  result["date"])
#print(result["description"])
print(result["text"])

#print(type(result))


China to launch hack-proof quantum communication network in 2016 ... Stephen Chen Updated ... 2014-11-04
China will complete and put into service the world's longest quantum communication network stretching 2,000km from Beijing to Shanghai by 2016, say scientists leading the project.
China to launch hack-proof quantum communication network in 2016
China will complete and put into service the world's longest quantum communication network stretching 2,000km from Beijing to Shanghai by 2016, say scientists leading the project.


In [ ]:
#print(result["comments"])
comments = re.split(r"Comment\s#[0-9]+\s", string=result["comments"])[1:]
print(len(comments))

comment_dicts = [{"date": datetime.strptime(re.sub(r"((st)|(nd)|(rd)|(th)),", "", cmt.split("\n")[0]), "%B %d %Y at %I:%M %p").date(), "body": " \n".join(cmt.split("\n")[1:])} for cmt in comments]
tl_comment_dicts = [cmt for cmt in comment_dicts if "#" not in "".join(cmt["body"].split("\n")[0:1]) ]

print(tl_comment_dicts[2]["body"])


In [4]:
from bs4 import BeautifulSoup

url_list = []
for page_n in range(0, 28+1):
    downloaded = fetch_url("https://scottaaronson.blog/?cat=4&paged=%d" % page_n)
    soup = BeautifulSoup(downloaded, 'html.parser')
    post_divs = soup.body.find(id="page").find(id="content").find_all(class_="post")
    post_ids = [int(div.h3["id"].split("-")[-1]) for div in post_divs]
    post_urls = ["https://scottaaronson.blog/?p=%d" % post_id for post_id in post_ids]
    url_list.extend(post_urls)

In [5]:
import pickle
with open("urllist_cache.pkl", "wb") as fp:   #Pickling
    pickle.dump(url_list, fp)

cached_url_list = []
with open("urllist_cache.pkl", "rb") as fp:   # Unpickling
    cached_url_list = pickle.load(fp)

In [7]:
import hashlib


def cmt_date_body_parse(cmt, url):
    try:
        return {"date": datetime.strptime(re.sub(r"((st)|(nd)|(rd)|(th)),", "", cmt.split("\n")[0]), "%B %d %Y at %I:%M %p").date(), "body": " \n".join(cmt.split("\n")[1:])}
    except:
        print(cmt,url)
        return None

num_urls = len(url_list)
article_instances = []
article_ids = []

for (iu, url) in enumerate(url_list):
    if validate_url(url) != None:
        if validate_url(url)[0]: #check whether url can be resolved or is damaged
            downloaded = fetch_url(url)
            result = bare_extraction(downloaded, 
                                        url=url, 
                                        include_comments=True,
                                        #include_formatting=False,
                                        favor_precision=True, #favor recall
                                        with_metadata=True,
                                        #deduplicate=True, 
                                        only_with_metadata=True,
                                        #url_blacklist=None,
                                        as_dict=True,
                                        )
            art_id = url.split("?p=")[-1]
            
            comments = re.split(r"\nComment\s#[0-9]+\s", string=result["comments"])[1:]
            comment_dicts = [cmt_date_body_parse(cmt, url) for cmt in comments] 
            comment_dicts = [cmt_dict for cmt_dict in comment_dicts if cmt_dict != None]
            tl_comment_dicts = [cmt for cmt in comment_dicts if "#" not in "".join(cmt["body"].split("\n")[0:1]) ]
            print(len(tl_comment_dicts))
            comment_instances = []
            comment_ids = []
            for cmt_dict in tl_comment_dicts:
                custom_cmt_id = hashlib.sha256(cmt_dict["body"].encode("utf-8")).hexdigest()
                cmt_inst = dm.Comment(comment_id=custom_cmt_id,
                            article_id=art_id,
                            date=cmt_dict["date"],
                            username="Unknown",
                            from_where="Shtetl-Optimized",
                            body=cmt_dict["body"],
                            level=0,
                            parent_comment_id=None,
                            replies=[],
                            replies_ids=[])
                comment_instances.append(cmt_inst)
                comment_ids.append(custom_cmt_id)
            
            art_inst = dm.Article(article_id=art_id,
                        article_type="blog_article",
                        link=url,
                        post_id=["N.A.", ],
                        posted_by=["N.A.", ],
                        date_posted=["N.A.", ],
                        post_title=["N.A.", ],
                        post_body=["N.A.", ],
                        date= result["date"],
                        author="Scott Aaronson",
                        headline=result["title"],
                        body=result["text"],
                        comments=comment_instances,
                        comments_ids=comment_ids)
            article_instances.append(art_inst)
            article_ids.append(art_id)
    print("Done with article %d of %d" % (iu, num_urls))

artset = dm.ArticleSet(label="Shtetl-Optimized: Quantum Category",
              as_list=article_instances,
              article_id_list=article_ids,)

filepath = "S:\\Sync\\University\\2023_MRP_1\\MRP1_WorkDir\\data\\"
artset.save_to_csv(filepath+"shtetl_optim")

49
Done with article 0 of 290
141
Done with article 1 of 290
32
Done with article 2 of 290
25
Done with article 3 of 290
6
Done with article 4 of 290
18
Done with article 5 of 290
20
Done with article 6 of 290
16
Done with article 7 of 290
25
Done with article 8 of 290
70
Done with article 9 of 290
49
Done with article 10 of 290
141
Done with article 11 of 290
32
Done with article 12 of 290
25
Done with article 13 of 290
6
Done with article 14 of 290
18
Done with article 15 of 290
20
Done with article 16 of 290
16
Done with article 17 of 290
25
Done with article 18 of 290
70
Done with article 19 of 290
92
Done with article 20 of 290
68
Done with article 21 of 290
As an immunologist that is interested in quantum computing I’m quite sure that I’m “not really good at anything”. I’m sure I can’t understand Shor’s algorythm – neither in 30 min nor 30 months, but what I do understand is that insulting colleagues on the internet is unprofessional and that your worldview has long since ceased 

In [1]:
#test newspaer package for url parsing:
from newspaper import Article

a = Article(url)
a.download()
a.parse()
#print(a.summary)
print(a.title, a.authors, a.publish_date)
print("Text: ", a.text)

#print(a.nlp())

NameError: name 'url' is not defined

In [ ]:
#test readabilipy package for url parsing:
import requests
from readabilipy import simple_json_from_html_string
req = requests.get(url)
article = simple_json_from_html_string(req.text, use_readability=True)
#print(article.keys())
print(article["title"], article["byline"], article["date"] )
print(article["plain_text"])

In [ ]:
import sys
import youtube_api as yapi
import pandas as pd
import json

import data_struct as dm


#test = yapi.video_search("quantum")

video_dict = {'youID':[], 'title':[], 'pub_date':[],'channel_name':[], "video_description":[]}
"""
just_json = test[1]

for video in just_json:
    print(video['id']['videoId'])
    print(video['snippet']['title'])
    print(video['snippet']['publishedAt'])
    print(video['snippet']['channelTitle'])
    print(video['snippet']['description'])
"""

def grab_videos(keyword, token=None):
    res = yapi.video_search(q=keyword, 
                         order="relevance",
                         token=token,
                         )
    c_token = res[0]
    c_videos = res[1]
    for vid in c_videos:
        video_dict['youID'].append(vid['id']['videoId'])
        video_dict['title'].append(vid['snippet']['title'])
        video_dict['pub_date'].append(vid['snippet']['publishedAt'])
    print("added ", str(len(c_videos)),  " videos to a total of ", str(len(video_dict['youID'])))
    return token


token = grab_videos("quantum")
while token != "last_page":
    token = grab_videos("quantum", token=token)


res = yapi.video_comments_query("Mz5gwDykgfA", token=None)
print(len(res[1]), len(res[2]))
